Check we have connection correct. To test this we call get a FHIR Conformance statement from the server

In [8]:
import requests
from fhirclient.models.fhirinstant import FHIRInstant
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()
import os

fhir_password = os.getenv("FHIR_PASSWORD")
fhir_username = os.getenv("FHIR_USERNAME")
#server = "https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/"
server = os.getenv("FHIR_SERVER")

api_url = server + "metadata"
print(api_url)
response = requests.get(api_url)
#response.json()

https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/metadata


Find a patient. Simple search for a patient named wrexham

In [9]:


api_url = server + "Patient?name=wrexham"
print(api_url)
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Patient?name=wrexham


{'resourceType': 'Bundle',
 'id': '7f826345-f420-4190-9274-e36d4202aee7',
 'type': 'searchset',
 'timestamp': '2025-12-25T08:34:31Z',
 'total': 0,
 'link': [{'relation': 'self',
   'url': 'https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Patient?name=wrexham'}]}

Now lets find diagnostic reports for this patient.

In [10]:
api_url = server + "DiagnosticReport?patient=18"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

{'resourceType': 'Bundle',
 'id': '38421c26-681f-4014-8a7d-857b5a3fe675',
 'type': 'searchset',
 'timestamp': '2025-12-25T08:34:36Z',
 'total': 0,
 'link': [{'relation': 'self',
   'url': 'https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/DiagnosticReport?patient=18'}]}

Can also return details about the organisation that created this.

In [12]:
api_url = server + "Organization"
print(api_url)
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response.json()

https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Organization


{'resourceType': 'Bundle',
 'id': '19ba753e-4bda-4fd8-8d9b-f987f64918b2',
 'type': 'searchset',
 'timestamp': '2025-12-25T08:35:03Z',
 'total': 15,
 'link': [{'relation': 'self',
   'url': 'https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Organization'}],
 'entry': [{'fullUrl': 'https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Organization/3',
   'resource': {'resourceType': 'Organization',
    'active': True,
    'id': '3',
    'identifier': [{'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
      'value': 'REP'}],
    'name': "LIVERPOOL WOMEN'S NHS FOUNDATION TRUST",
    'partOf': {'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code',
      'value': 'QYG'},
     'reference': 'Organization/8'},
    'type': [{'coding': [{'code': 'RO197',
        'display': 'NHS TRUST',
        'system': 'https://digital.nhs.uk/services/organisation-data-service/CodeSystem/ODSOrganisationRole'}]},
     {'coding': [{'code': 'RO57',
        'displa

Example using python to interpret the json and also using [fhirclient](https://github.com/smart-on-fhir/client-py)

In [13]:
import fhirclient.models.diagnosticreport as dr
import pandas as pd

api_url = server + "DiagnosticReport?_count=50"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response1JSON = response.json()

print(response1JSON['total'])

reports = []
for entry in response1JSON['entry']:
    #print(entry['resource'])
    print(entry['resource']['resourceType'], entry['resource']['issued'] )
    report = dr.DiagnosticReport(entry['resource'])
    reports.append(report)
    for coding in report.code.coding:
        print(coding.code)

10
DiagnosticReport 2025-12-18T10:13:19+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-11-21T12:05:26+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-11-21T12:22:53+00:00
CCP19v1_R112
R112.1
R115.1
DiagnosticReport 2025-11-21T12:42:05+00:00
CCP19v1_R112
R112.1
R115.1
DiagnosticReport 2025-11-21T12:53:09+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-11-21T13:47:18+00:00
CCP19v1_R112
R112.1
R115.1
DiagnosticReport 2025-11-21T14:01:59+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-11-21T15:34:09+00:00
CCP19v1_R112
R112.1
R115.1
DiagnosticReport 2025-11-25T14:15:15+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-11-27T10:29:01+00:00
ctDNA_M4
M4.14


We are aiming at display graphs based on the order and the report.

We can include the order which in FHIR is held in ServiceRequest by including this in the query.

In [15]:
import fhirclient.models.servicerequest as sr

api_url = server + "DiagnosticReport?_include=DiagnosticReport:based-on"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
responseInclude = response.json()

#print(responseInclude)
print(responseInclude['total'])

serviceRequests = []
diagnosticReports = []
for entry in responseInclude['entry']:

    if entry['resource']['resourceType'] == 'DiagnosticReport':
        report = dr.DiagnosticReport(entry['resource'])
        diagnosticReports.append(report)
    if entry['resource']['resourceType'] == 'ServiceRequest':
        request = sr.ServiceRequest(entry['resource'])
        serviceRequests.append(request)

print("ServiceRequest = " + str(len(serviceRequests)))
print("DiagnosticReport = " + str(len(diagnosticReports)))

10
ServiceRequest
ServiceRequest
ServiceRequest
ServiceRequest
ServiceRequest
ServiceRequest
ServiceRequest
ServiceRequest
ServiceRequest
ServiceRequest = 9
DiagnosticReport = 10


Process the diaganostic reports

Process the reports, convert some of the objects into other datatypes.

In [16]:
def performer(my_list):
    performr = ""
    if my_list != None:
        for item in my_list:
            performr = item.display
    return performr

def performerCode(my_list):
    performr = None
    if my_list != None:
        for item in my_list:
            performr = item.identifier.value
    return performr

def codeCode(concept):
    code = ""
    for coding in concept.coding:
        code = coding.code

    return code
def codeDisplay(concept):
    code = ""
    for coding in concept.coding:
        code = coding.display

    return code

def issued(issued):
    if issued == None:
        return None
    return issued.isostring

def serviceRequest(my_list):
    sr = None
    if my_list != None:
        for item in my_list:
            if item.reference != None:
                sr = item.reference.replace('ServiceRequest/', '')
    return sr

print(len(diagnosticReports))
dfDR = pd.DataFrame([vars(s) for s in diagnosticReports])

dfDR['performerDisplay'] = dfDR['performer'].apply(performer)
dfDR['performerCode'] = dfDR['performer'].apply(performerCode)
dfDR['codingCode'] = dfDR['code'].apply(codeCode)
dfDR['codingDisplay'] = dfDR['code'].apply(codeDisplay)
dfDR['issuedDate'] = dfDR['issued'].apply(issued)
dfDR['effectiveDate'] = dfDR['effectiveDateTime'].apply(issued)
dfDR['serviceRequestId'] = dfDR['basedOn'].apply(serviceRequest)

dfDiagnosticReport = dfDR[['id','performerDisplay','performerCode','codingCode', 'codingDisplay', 'issuedDate', 'effectiveDate', 'serviceRequestId']]
dfDiagnosticReport

10


,id,performerDisplay,performerCode,codingCode,codingDisplay,issuedDate,effectiveDate,serviceRequestId
0,6,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-18T10:13:19+00:00,2025-12-18T10:13:19+00:00,81
1,16,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-11-21T12:05:26+00:00,2025-11-21T12:05:26+00:00,None
2,21,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-11-21T12:22:53+00:00,2025-11-21T12:22:53+00:00,74
3,27,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-11-21T12:42:05+00:00,2025-11-21T12:42:05+00:00,76
4,31,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-11-21T12:53:09+00:00,2025-11-21T12:53:09+00:00,30
5,39,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-11-21T13:47:18+00:00,2025-11-21T13:47:18+00:00,36
6,43,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-11-21T14:01:59+00:00,2025-11-21T14:01:59+00:00,42
7,49,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-11-21T15:34:09+00:00,2025-11-21T15:34:09+00:00,83
8,53,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-11-25T14:15:15+00:00,2025-11-25T14:15:15+00:00,52
9,61,NHS North West Genomics,699X0,M4.14,"Non-Small Cell Lung Cancer, Multi-target ctDNA...",2025-11-27T10:29:01+00:00,2025-11-27T10:29:01+00:00,87


Clean up requests

In [17]:
print(len(serviceRequests))

def requester(item):
    performr = None
    if item != None:
        performr = item.display
    return performr

def requesterCode(item):
    performr = None
    if item != None:
        performr = item.identifier.value
    return performr

dfSR = pd.DataFrame([vars(s) for s in serviceRequests])
dfSR['requesterDisplay'] = dfSR['requester'].apply(requester)
dfSR['requesterCode'] = dfSR['requester'].apply(requesterCode)
dfSR['authoredOnDate'] = dfSR['authoredOn'].apply(issued)

dfServiceRequest = dfSR[['id','requesterDisplay','requesterCode','authoredOnDate']]
dfServiceRequest

9


,id,requesterDisplay,requesterCode,authoredOnDate
0,74,Bolton NHS Foundation Trust,RMC,2025-11-06T00:00:00
1,76,Stockport NHS Foundation Trust,RWJ,2025-11-06T00:00:00
2,30,Manchester University NHS Foundation Trust,R0A,2025-11-06T00:00:00
3,36,Manchester University NHS Foundation Trust,R0A,2025-11-06T00:00:00
4,42,Manchester University NHS Foundation Trust,R0A,2025-11-06T00:00:00
5,83,Bolton NHS Foundation Trust,RMC,2025-11-06T00:00:00
6,52,Manchester University NHS Foundation Trust,R0A,2025-11-03T00:00:00
7,81,Liverpool Women's NHS Foundation Trust,REP,2025-11-06T00:00:00
8,87,Unknown,UNK,2025-11-05T00:00:00


Join both dataframes into a single result.

In [18]:
# ... existing code ...
df = pd.merge(
    dfDiagnosticReport,
    dfServiceRequest,
    left_on='serviceRequestId',
    right_on='id',
    how="left",
    indicator=True,
    suffixes=('_dr', '_sr')
)

df

,id_dr,performerDisplay,performerCode,codingCode,codingDisplay,issuedDate,effectiveDate,serviceRequestId,id_sr,requesterDisplay,requesterCode,authoredOnDate,_merge
0,6,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-18T10:13:19+00:00,2025-12-18T10:13:19+00:00,81,81,Liverpool Women's NHS Foundation Trust,REP,2025-11-06T00:00:00,both
1,16,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-11-21T12:05:26+00:00,2025-11-21T12:05:26+00:00,None,NaN,NaN,NaN,NaN,left_only
2,21,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-11-21T12:22:53+00:00,2025-11-21T12:22:53+00:00,74,74,Bolton NHS Foundation Trust,RMC,2025-11-06T00:00:00,both
3,27,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-11-21T12:42:05+00:00,2025-11-21T12:42:05+00:00,76,76,Stockport NHS Foundation Trust,RWJ,2025-11-06T00:00:00,both
4,31,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-11-21T12:53:09+00:00,2025-11-21T12:53:09+00:00,30,30,Manchester University NHS Foundation Trust,R0A,2025-11-06T00:00:00,both
5,39,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-11-21T13:47:18+00:00,2025-11-21T13:47:18+00:00,36,36,Manchester University NHS Foundation Trust,R0A,2025-11-06T00:00:00,both
6,43,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-11-21T14:01:59+00:00,2025-11-21T14:01:59+00:00,42,42,Manchester University NHS Foundation Trust,R0A,2025-11-06T00:00:00,both
7,49,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-11-21T15:34:09+00:00,2025-11-21T15:34:09+00:00,83,83,Bolton NHS Foundation Trust,RMC,2025-11-06T00:00:00,both
8,53,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-11-25T14:15:15+00:00,2025-11-25T14:15:15+00:00,52,52,Manchester University NHS Foundation Trust,R0A,2025-11-03T00:00:00,both
9,61,NHS North West Genomics,699X0,M4.14,"Non-Small Cell Lung Cancer, Multi-target ctDNA...",2025-11-27T10:29:01+00:00,2025-11-27T10:29:01+00:00,87,87,Unknown,UNK,2025-11-05T00:00:00,both
